# DSC212: Modularity on the Karate Club Graph
**Name:** Emylin Mary Samuval  
**Roll Number:** IMS24090

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.linalg import eigh
from collections import defaultdict
import os

SEED = 42
np.random.seed(SEED)

OUTDIR = "."  # Save images in root


In [ ]:
def modularity_matrix(G):
    nodes = list(G.nodes())
    A = nx.to_numpy_array(G, nodelist=nodes)
    k = A.sum(axis=1)
    m = A.sum()/2
    P = np.outer(k, k) / (2*m)
    return A - P, nodes

def restricted(B, nodes_full, sub):
    idx = [nodes_full.index(v) for v in sub]
    return B[np.ix_(idx, idx)]

def spectral_split(Bc, nodes):
    w, v = eigh(Bc)
    u = v[:, -1]
    pos = [nodes[i] for i in range(len(nodes)) if u[i] > 0]
    neg = [nodes[i] for i in range(len(nodes)) if u[i] <= 0]
    return w[-1], pos, neg


In [ ]:
def recursive_bisect(G):
    B_full, nodes_full = modularity_matrix(G)
    queue = [list(G.nodes())]
    final = []
    history = [queue.copy()]
    while queue:
        C = queue.pop(0)
        Bc = restricted(B_full, nodes_full, C)
        lam, pos, neg = spectral_split(Bc, C)
        if lam <= 1e-12 or len(pos) == 0 or len(neg) == 0:
            final.append(C)
        else:
            queue.insert(0, pos)
            queue.insert(0, neg)
        history.append(queue + final)
    return final, history

G = nx.karate_club_graph()
communities, history = recursive_bisect(G)
communities


In [ ]:
pos = nx.spring_layout(G, seed=SEED)
cmap = cm.get_cmap('tab20')

def stepwise(G):
    B_full, nodes_full = modularity_matrix(G)
    parts = [[list(G.nodes())]]
    q = [list(G.nodes())]
    done = []
    while q:
        C = q.pop(0)
        Bc = restricted(B_full, nodes_full, C)
        lam, posC, negC = spectral_split(Bc, C)
        if lam <= 1e-12 or len(posC) == 0 or len(negC) == 0:
            done.append(C)
        else:
            cur = parts[-1].copy()
            for i, b in enumerate(cur):
                if set(b) == set(C):
                    idx = i
                    break
            cur.pop(idx)
            cur.insert(idx, posC)
            cur.insert(idx+1, negC)
            parts.append(cur)
            q.insert(0, posC)
            q.insert(0, negC)
    return parts

parts = stepwise(G)

for it, part in enumerate(parts):
    mapping = {}
    for i, c in enumerate(part):
        for n in c:
            mapping[n] = i
    colors = [mapping[n] for n in G.nodes()]
    plt.figure(figsize=(6,5))
    nx.draw_networkx(G, pos, node_color=colors, cmap=cmap, node_size=300, labels={n:n for n in G.nodes()})
    plt.axis("off")
    plt.savefig(f"iter_{it}.png")
    plt.close()


In [ ]:
metric_hist = defaultdict(list)
iters = list(range(len(parts)))

for it in iters:
    deg = nx.degree_centrality(G)
    bet = nx.betweenness_centrality(G)
    clo = nx.closeness_centrality(G)
    clu = nx.clustering(G)
    for n in G.nodes():
        metric_hist[("deg", n)].append(deg[n])
        metric_hist[("bet", n)].append(bet[n])
        metric_hist[("clo", n)].append(clo[n])
        metric_hist[("clu", n)].append(clu[n])

for key in ["deg", "bet", "clo", "clu"]:
    plt.figure(figsize=(10,6))
    for n in G.nodes():
        plt.plot(iters, metric_hist[(key, n)])
    plt.xlabel("Iteration")
    plt.ylabel(key)
    plt.savefig(f"{key}_evol.png")
    plt.close()


### Discussion
Nodes like 0 and 33 remain central because they act as hubs or bridges. Degree values remain constant since the graph does not change. Betweenness and closeness highlight bridging roles between emerging communities. Clustering is higher for tightly connected groups, revealing internal community structure.